In [11]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 벡터저장소 검색 메모리(VectorStoreRetrieverMemory)

In [2]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [3]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()

# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory

# 관련성 있는 정보 k개를 반환
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
memory = VectorStoreRetrieverMemory(retriever=retriever)

임의의 대화 저장

In [ ]:
# history 키에 대화내용이 저장됨
memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)
memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

In [9]:
print(memory.load_memory_variables({"prompt":"면접자의 전공은?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.
human: 개발자로서 자신의 강점은 무엇이라고 생각하나요?
ai: 제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다.


In [10]:
print(memory.load_memory_variables({"prompt":"면접자의 열할?"})["history"])

human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.
human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.


# 벡터 스토어 종류

## 왜 벡터 스토어가 필요한지?
1. 의미가 비슷한 벡터를 유사도 기반으로 검색 가능! (단순 키워드 검색보다 성능 높음)
2. 일반 RDB나 NoSQL보다 속도 빠름

In [12]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("3-2.추가학습")

LangSmith 추적을 시작합니다.
[프로젝트명]
3-2.추가학습


## 1. Chroma

샘플 데이터 업로드

In [13]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
path = os.getcwd()

loader1 = TextLoader(f"{path}/data/nlp-keywords.txt")
loader2 = TextLoader(f"{path}/data/finance-keywords.txt")

split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

len(split_doc1), len(split_doc2)

(11, 6)

In [ ]:
chroma_db = Chroma.from_documents(
    documents=split_doc1, embedding=OpenAIEmbeddings(),collection_name="my_db"
)
# 단순히 text를 임베딩 하고 싶을 떄 :
# db2 = Chroma.from_texts(
#     ["안녕하세요. 정말 반갑습니다.", "제 이름은 테디입니다."],
#     embedding=OpenAIEmbeddings(),
# )

In [15]:
# 저장 경로
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장
persist_db = Chroma.from_documents(
    split_doc1, OpenAIEmbeddings(),persist_directory=DB_PATH, collection_name="my_db"
)

In [16]:
# 디스크에서 문서를 로드
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(),
    collection_name="my_db"
)

In [17]:
# 저장된 데이터 확인
persist_db.get()

{'ids': ['f3ee4bf4-e8ca-46e1-9631-b89b9efbdae7',
  '055a2eb6-49d5-40df-ac03-f1b3b657f777',
  '218e873a-a4a2-4b87-81db-53debffefc40',
  '4ef826d6-16fb-4073-8210-bcb16596ca7c',
  '5cdd0f1b-b1a5-4c9e-a717-e845ab9dd608',
  '9d1af28e-cb48-45ca-b627-5794eb69dacf',
  'dfae701f-102d-4938-8025-722c4e16d40b',
  '7a89b306-9218-4f15-8832-4953f25a058b',
  '46c911ff-df3e-4c05-872e-e05b80685ea2',
  'e89ff074-9573-4491-8138-67e78f2a6868',
  '11822778-51e1-48a8-9ee0-34d73a8d57ea'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

유사도 검색

In [19]:
chroma_db.similarity_search("TF IDF에 대해 알려줘", k=2)

[Document(id='ccbc5563-d845-4839-9452-b75b6a3c08a3', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='732409a1-144d-43ea-a521-988eb78e9663', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 

In [23]:
chroma_db.similarity_search(
    "TF IDF에 대해 알려줘",
    filter={"source":f"{path}/data/finance_keywords.txt"},
    k=2
)

[]

벡터 저장소에 문서 추가(add_documents)

In [26]:
from langchain_core.documents import Document

chroma_db.add_documents(
    [
        Document(
            page_content="도큐먼트를 새로 추가해 보겠습니다",
            metadata = {"source":"mydata.txt"},
            id = "1",
        )
    ]
)

['1']

In [27]:
chroma_db.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['도큐먼트를 새로 추가해 보겠습니다'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

벡터 저장소를 검색기(retriever)로 변환

In [28]:
db = Chroma.from_documents(
    documents=split_doc1+split_doc2,
    embedding=OpenAIEmbeddings(),
    collection_name="nlp",
)

In [29]:
retriever = db.as_retriever()
retriever.invoke("Word2Vec에 대해 알려줘",k=2)

[Document(id='29cc2962-e58c-4cd9-a486-bde6f70f99fa', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='d5bdae14-33a6-4f4b-8539-82e200befd8d', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 

MMR (Maximal Marginal Relevance):
- 검색된 문서 간 다양성(diversity)을 고려하여 중복이 적은 결과를 반환

In [30]:
retriever = db.as_retriever(
    search_type="mmr", # mmr기반 검색 수행
    search_kwargs={
        "k": 6, 
        "lambda_mult": 0.25, # 유사성과 다양성 간의 균형 (0이면 다양성만, 1이면 유사성만)
        "fetch_k": 10, # 후보 문서를 몇 개까지 먼저 가져올지
        "score_threshold": 0.8} # 특정 임계값 이상의 유사도
)
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='29cc2962-e58c-4cd9-a486-bde6f70f99fa', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='bd2c952e-60b2-4a12-837b-0356a1abb5a9', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의

# 2. FAISS

- Faiss는 고성능 벡터 검색 및 클러스터링을 위한 라이브러리
- 임베뎅 함수, faiss 인덱스, 문서 저장소를 조합하여 효율적인 검색 시스템 구축 가능

In [32]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1536


In [ ]:
# FAISS 벡터 저장소 생성 (직접구성)
db = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# DB 생성 (간편생성))
db = FAISS.from_documents(documents=split_doc1, embedding=OpenAIEmbeddings())

In [35]:
# 문서 저장소 ID 확인
db.index_to_docstore_id

{0: 'e9578921-a131-4356-b90a-023012f4aef0',
 1: 'a172b581-7cca-4cdc-9839-74e70d85d3e6',
 2: '018b0254-156c-494c-af2d-4cb1772f005d',
 3: '89cb5df1-4996-41df-a41f-b08742352c8b',
 4: 'da482606-8d4d-4f7f-a6dc-75ad054baa64',
 5: 'fac17fa5-3d5d-42fa-b5ea-af511b75bc37',
 6: 'e5de82d4-4899-4355-84c0-d147b1141f84',
 7: '247c641c-eeb3-4113-8838-d653e7026912',
 8: '64c82176-93a8-4cef-aa1e-edba4769bae4',
 9: 'e106a9f5-09ac-40ed-a9cd-a55fcdb10ef3',
 10: '635fca4d-b470-41dd-b04d-d3e3aa029b83'}

In [36]:
# 저장된 문서의 ID: Document 확인
db.docstore._dict

{'e9578921-a131-4356-b90a-023012f4aef0': Document(id='e9578921-a131-4356-b90a-023012f4aef0', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer'),
 'a172b581-7cca-4cdc-9839-74e70d85d3e6': Document(id='a172b581-7cca-4cdc-9839-74e70d85d3e6', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는

유사도 검색

In [ ]:

db.similarity_search("TF IDF 에 대하여 알려줘", k=2)

[Document(id='e5de82d4-4899-4355-84c0-d147b1141f84', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='fac17fa5-3d5d-42fa-b5ea-af511b75bc37', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 

문서로부터 추가 (add_documents)

In [38]:
from langchain_core.documents import Document

# page_content, metadata 지정
db.add_documents(
    [
        Document(
            page_content="도큐먼트를 새로 추가해 볼께요",
            metadata={"source": "mydata.txt"},
        )
    ],
    ids=["new_doc1"],
)

['new_doc1']

In [39]:
# 추가된 데이터를 확인
db.similarity_search("안녕하세요", k=1)

[Document(id='new_doc1', metadata={'source': 'mydata.txt'}, page_content='도큐먼트를 새로 추가해 볼께요')]

저장 및 로드

In [41]:
# 로컬 Disk 에 저장
db.save_local(folder_path="faiss_db", index_name="faiss_index")

In [ ]:
# 로컬에 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [43]:
# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: 'e9578921-a131-4356-b90a-023012f4aef0',
 1: 'a172b581-7cca-4cdc-9839-74e70d85d3e6',
 2: '018b0254-156c-494c-af2d-4cb1772f005d',
 3: '89cb5df1-4996-41df-a41f-b08742352c8b',
 4: 'da482606-8d4d-4f7f-a6dc-75ad054baa64',
 5: 'fac17fa5-3d5d-42fa-b5ea-af511b75bc37',
 6: 'e5de82d4-4899-4355-84c0-d147b1141f84',
 7: '247c641c-eeb3-4113-8838-d653e7026912',
 8: '64c82176-93a8-4cef-aa1e-edba4769bae4',
 9: 'e106a9f5-09ac-40ed-a9cd-a55fcdb10ef3',
 10: '635fca4d-b470-41dd-b04d-d3e3aa029b83',
 11: 'new_doc1'}

In [44]:
# 새로운 FAISS 벡터 저장소 생성
db2 = FAISS.from_documents(documents=split_doc2, embedding=OpenAIEmbeddings())
db2.index_to_docstore_id


{0: 'cd41ee3b-3724-4918-8ec1-ef6a06975813',
 1: '15ae8c7e-9825-4a21-8017-16f8e088d94a',
 2: '6c340430-45c0-476b-9675-80f711fe2203',
 3: '60a208e0-371a-43dd-8cd8-1a7c6aa44471',
 4: 'bececb18-62cf-42db-bbb7-4fbc80aba3cc',
 5: '2e19a04f-03c8-4764-8d9a-2265c8549beb'}

In [45]:
# db + db2 를 병합
db.merge_from(db2)

# 병합된 데이터 확인
db.index_to_docstore_id


{0: 'e9578921-a131-4356-b90a-023012f4aef0',
 1: 'a172b581-7cca-4cdc-9839-74e70d85d3e6',
 2: '018b0254-156c-494c-af2d-4cb1772f005d',
 3: '89cb5df1-4996-41df-a41f-b08742352c8b',
 4: 'da482606-8d4d-4f7f-a6dc-75ad054baa64',
 5: 'fac17fa5-3d5d-42fa-b5ea-af511b75bc37',
 6: 'e5de82d4-4899-4355-84c0-d147b1141f84',
 7: '247c641c-eeb3-4113-8838-d653e7026912',
 8: '64c82176-93a8-4cef-aa1e-edba4769bae4',
 9: 'e106a9f5-09ac-40ed-a9cd-a55fcdb10ef3',
 10: '635fca4d-b470-41dd-b04d-d3e3aa029b83',
 11: 'new_doc1',
 12: 'cd41ee3b-3724-4918-8ec1-ef6a06975813',
 13: '15ae8c7e-9825-4a21-8017-16f8e088d94a',
 14: '6c340430-45c0-476b-9675-80f711fe2203',
 15: '60a208e0-371a-43dd-8cd8-1a7c6aa44471',
 16: 'bececb18-62cf-42db-bbb7-4fbc80aba3cc',
 17: '2e19a04f-03c8-4764-8d9a-2265c8549beb'}

검색기로 변환

In [46]:
# 새로운 FAISS 벡터 저장소 생성
db = FAISS.from_documents(
    documents=split_doc1 + split_doc2, embedding=OpenAIEmbeddings()
)

In [ ]:
# MMR 검색 수행
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2, "lambda_mult": 0.25, "fetch_k": 10}
)
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='de88c096-787a-4e1e-97fb-72c9fe0e48df', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='f788fae3-8e79-49c1-9221-ab8d8e5c77c5', metadata={'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으